In [2]:
from functools import reduce
import pandas as pd
import numpy as np
from tabulate import tabulate

In [20]:
def nim_sum(vals):
    """ Calculates the NIM-sum for a collection of values """
    return reduce(lambda a, b: a ^ b, vals)

In [9]:
class Game:
    def __init__(self, game_type, start_pos):
        self.game_type = game_type
        self.pos = start_pos
        
class GameSum:
    """ Calculates the next best move for a sum of games """
    
    def __init__(self, game_calculators):
        self.game_calculators = game_calculators
        
    def best_move(self, games):
        """ 
        Determines the next best move for a set of games
        games: collection of Games 
        returns: collection of Games with the best legal move made
        """
        
        games_grundy = [self.game_calculators[game.game_type].get_grundy(game.pos) for game in games]
        sum_grundy = nim_sum(games_grundy) # Grundy value for all games is nim sum of grundy values for each game
        
        if sum_grundy == 0:
            # Any move from here is a losing move. Determine best move, TODO
            print("Not implemented")
            return games
        
        for i, game in enumerate(games):
            desired_value = games_grundy[i] ^ sum_grundy # Achieving this grundy value for the game would be a winning move
            if desired_value < games_grundy[i]:
                # Desired value less than the current position's grundy value, some follower guaranteed to have this value
                followers = self.game_calculators[game.game_type].followers(game.pos)
                best_move = next(f for f in followers if self.game_calculators[game.game_type].get_grundy(f) == desired_value)
                game.pos = best_move
                return games
                
        raise Exception("No best move found for game sum")
    
    def is_terminal(self, games):
        """ 
        Returns true if a position is terminal 
        In a sum of games, this is true if every game is terminal
        """
        return all (self.game_calculators[game.game_type].is_terminal(game.pos) for game in games)
    
    def visualize_game(self, games):
        """
        Prints visualization of sum of games to console
        """
        df = pd.DataFrame()
        visualize_arrs = [self.game_calculators[game.game_type].visualize_game_arr(game.pos) for game in games]
        max_lines = max([len(arr) for arr in visualize_arrs])
        padded_arrs = [arr + [''] * (max_lines - len(arr)) for arr in visualize_arrs]
        for i, arr in enumerate(padded_arrs):
            df.insert(i, f'Game {i+1}', arr)
        
        print(tabulate(df, headers=df.columns, showindex=False))
    
    def parse_move(self, games, inp):
        """ 
        Parses a user input move to a position
        games: Current games in the sum, note that this array WILL be modified to the new position 
        inp: Player inputted move of the form: <game_no> : <game_pos> 
        Returns the new position if valid, otherwise returns false
        """
        i,move_inp = inp.split(':', 1)
        i = int(i) # Game number to move in
        if not i in range(1,len(games)+1):
            print(f'Invalid game number {i}, valid range is {1} to {len(games)}')
            return False
        
        game_calc = self.game_calculators[games[i-1].game_type]
        parsed_move = game_calc.parse_move(games[i-1].pos, move_inp)
        
        if not parsed_move:
            print(f'Invalid move in game of type {games[i-1].game_type}')
            print(f'Original position: {games[i-1].pos}, inputted position: {move_inp}')
            return False
        
        games[i-1].pos = parsed_move
        return games
        